## How To Assign Training Parameters to a Quantum Kernel

In this guide, we show the ins and outs of assigning training parameters to a `TrainableKernel` instance using Qiskit Machine Learning. 

We can create a `TrainableFidelityQuantumKernel` (`QK`) and specify our feature map and trainable parameters. This can be done at initialization by passing an array of `Parameters` as the `training_parameters` argument to the `QK` constructor.

After the `QK.training_parameters` field has been set, `QK.assign_training_parameters()` offers two ways to assign values to the training parameters

1. Bind training parameters using a dictionary
    - Keys to dict must be parameters within the feature map and must exist in `QK.training_parameters`
    - Values in dict may be either numerical assignments or `ParameterExpression` objects
2. Bind user parameters using a list of values
    - If binding using a list of values, the list must be of same size and ordering as `QK.training_parameters`. Each input value will be bound to its corresponding ``training_parameters`` value.
     
We begin by importing a few packages and instantiating a feature map circuit with three trainable parameters, `θ`, and three input parameters, `x`.

In [1]:
# pylint: disable=import-error, wrong-import-position, pointless-statement
import os
import sys
import numpy as np

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel

In [2]:
NUM_QUBITS = 3
fm = QuantumCircuit(NUM_QUBITS)
input_params = ip = ParameterVector("x", NUM_QUBITS)
training_params = tp = ParameterVector("θ", NUM_QUBITS)

for i in range(NUM_QUBITS):
    fm.h(i)
    fm.ry(tp[i], i)

for i in range(NUM_QUBITS):
    fm.crx(ip[i], (i) % NUM_QUBITS, (i + 1) % NUM_QUBITS)

# Define a Quantum Kernel using our trainable feature map
qk = TrainableFidelityQuantumKernel(
    feature_map=fm, training_parameters=training_params[:NUM_QUBITS]
)

print("input_params:", input_params)
print("training_params:", training_params)
qk.feature_map.draw()

input_params: x, ['x[0]', 'x[1]', 'x[2]']
training_params: θ, ['θ[0]', 'θ[1]', 'θ[2]']


┌───┐┌──────────┐                        ┌──────────┐
q_0: ┤ H ├┤ Ry(θ[0]) ├─────■──────────────────┤ Rx(x[2]) ├
     ├───┤├──────────┤┌────┴─────┐            └────┬─────┘
q_1: ┤ H ├┤ Ry(θ[1]) ├┤ Rx(x[0]) ├─────■───────────┼──────
     ├───┤├──────────┤└──────────┘┌────┴─────┐     │      
q_2: ┤ H ├┤ Ry(θ[2]) ├────────────┤ Rx(x[1]) ├─────■──────
     └───┘└──────────┘            └──────────┘

### Option  1: Bind User Parameters with a Dictionary

Here, we will use a dictionary of the form `{Parameter : Value}` that maps training parameters to either numeric values or `ParameterExpression` objects.

In [3]:
# Bind parameters to numeric values
param_binds = {tp[0]: np.pi / 2, tp[1]: np.pi / 3, tp[2]: np.pi / 4}

qk.assign_training_parameters(param_binds)
qk.parameter_values

array([1.57079633, 1.04719755, 0.78539816])

We are free to bind a subset of our training parameters and re-bind parameters to new values.

In [4]:
# Create incomplete training param bindings
param_binds = {tp[0]: np.pi / 6, tp[1]: np.pi / 5}

qk.assign_training_parameters(param_binds)
qk.parameter_values

array([0.52359878, 0.62831853, 0.78539816])

We can  un-bind our training parameters or assign training parameters to different `ParameterExpression` objects. This is done in in the same way that we would bind numeric values. 

In [5]:
# Create incomplete user param bindings
param_binds = {tp[0]: tp[0], tp[1]: tp[0] + tp[2], tp[2]: tp[2]}

qk.assign_training_parameters(param_binds)
qk.parameter_values

array([ParameterVectorElement(θ[0]), ParameterExpression(θ[0] + θ[2]),
       ParameterVectorElement(θ[2])], dtype=object)

### Option 2: Bind Training Parameters with a List

If the `training_parameters` have been specified in the `QuantumKernel`, we may bind and unbind those parameters using only lists of parameter values. Note that the list of values must always be equal in size to the `QuantumKernel.training_parameters` array, and the values will be assigned in order.

Here we instantiate a new quantum kernel with the three training parameters unbound.

In [6]:
qk = TrainableFidelityQuantumKernel(feature_map=fm, training_parameters=training_params)
qk.feature_map.draw()

┌───┐┌──────────┐                        ┌──────────┐
q_0: ┤ H ├┤ Ry(θ[0]) ├─────■──────────────────┤ Rx(x[2]) ├
     ├───┤├──────────┤┌────┴─────┐            └────┬─────┘
q_1: ┤ H ├┤ Ry(θ[1]) ├┤ Rx(x[0]) ├─────■───────────┼──────
     ├───┤├──────────┤└──────────┘┌────┴─────┐     │      
q_2: ┤ H ├┤ Ry(θ[2]) ├────────────┤ Rx(x[1]) ├─────■──────
     └───┘└──────────┘            └──────────┘

We may want to assign numerical values to parameters 0 and 2, while leaving parameter 1 unchanged.

In [7]:
param_values = [np.pi / 7, tp[1], np.pi / 9]
qk.assign_training_parameters(param_values)
qk.parameter_values

array([0.4487989505128276, ParameterVectorElement(θ[1]),
       0.3490658503988659], dtype=object)

To assign parameter 1 to a numerical value, while leaving parameters 0 and 2 unchaged, we pass in a full list of the new values (values 0 and 2 will remain the same.)

In [8]:
param_values = [np.pi / 7, np.pi / 6, np.pi / 9]
qk.assign_training_parameters(param_values)
qk.parameter_values

array([0.44879895, 0.52359878, 0.34906585])

Finally, if we want to unbind all of our parameters, we may just pass in a list of the parameters themselves.

In [9]:
param_values = [tp[0], tp[1], tp[2]]
qk.assign_training_parameters(param_values)
qk.parameter_values

array([ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1]),
       ParameterVectorElement(θ[2])], dtype=object)

In [10]:
import qiskit.tools.jupyter

%qiskit_version_table
%qiskit_copyright